# Neuroscience Quick Start: fMRI Brain State Classification

**Duration:** 10-30 minutes  
**Goal:** Classify brain states from fMRI activity patterns using machine learning

## What You'll Learn

- Load and explore fMRI brain imaging data (NIfTI format)
- Visualize brain activity patterns
- Extract features from BOLD signals
- Train classifiers to decode cognitive states
- Identify discriminative brain regions

## Dataset

We'll use the **Haxby 2001** dataset:
- Classic fMRI study on visual object recognition
- 8 object categories (faces, houses, cats, bottles, scissors, shoes, chairs, scrambled)
- 1 subject, 12 runs
- Pre-processed BOLD fMRI data
- Source: Nilearn datasets

No AWS account or API keys needed - let's get started!

## 1. Setup and Data Loading

In [ ]:
# Import libraries (all pre-installed in Colab/Studio Lab)
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.svm import SVC

warnings.filterwarnings("ignore")

# Neuroimaging libraries
import nibabel as nib
from nilearn import datasets, image, plotting
from nilearn.maskers import NiftiMasker
from nilearn.plotting import show

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 11

print("✓ Libraries loaded successfully!")
print("  - NiBabel for NIfTI file handling")
print("  - Nilearn for fMRI analysis and visualization")

In [ ]:
# Load the Haxby dataset (downloads ~30MB on first run)
print("Downloading Haxby 2001 dataset...")
print("(This may take 1-2 minutes on first run)\n")

haxby_dataset = datasets.fetch_haxby()

# Get file paths
fmri_filename = haxby_dataset.func[0]  # Functional data (4D: x, y, z, time)
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
mask_filename = haxby_dataset.mask_vt[0]  # Visual cortex mask

print("✓ Dataset loaded successfully!")
print(f"  fMRI data: {fmri_filename}")
print(f"  Brain mask: {mask_filename}")
print(f"  Labels shape: {labels.shape}")
print(f"\nNumber of volumes: {len(labels)}")
print(f"Stimulus categories: {labels['labels'].unique()}")

### Understanding fMRI Data

**fMRI (functional Magnetic Resonance Imaging):**
- Measures brain activity via blood oxygen level-dependent (BOLD) signal
- 4D data: 3D brain volume + time
- Each volume = snapshot of whole-brain activity
- Spatial resolution: ~2-3mm
- Temporal resolution: ~2 seconds (TR = repetition time)

**Brain Decoding:**
- Goal: Predict what someone is seeing/thinking from brain activity
- Method: Machine learning on spatial patterns of activation

## 2. Data Exploration and Visualization

In [ ]:
# Load the fMRI image to inspect dimensions
fmri_img = nib.load(fmri_filename)
fmri_data = fmri_img.get_fdata()

print("=== fMRI Data Dimensions ===")
print(f"Full 4D shape: {fmri_data.shape}")
print(f"  Spatial (x, y, z): {fmri_data.shape[:3]}")
print(f"  Temporal (volumes): {fmri_data.shape[3]}")
print(f"  Voxel size: {fmri_img.header.get_zooms()[:3]} mm")
print(f"  TR (repetition time): {fmri_img.header.get_zooms()[3]} seconds")
print(f"\nTotal number of voxels: {np.prod(fmri_data.shape[:3]):,}")

In [ ]:
# Visualize the brain anatomy (mean over time)
mean_img = image.mean_img(fmri_img)

print("Displaying mean fMRI volume (anatomical reference):\n")
plotting.plot_anat(
    mean_img,
    title="Mean fMRI Volume (Anatomical Reference)",
    cut_coords=(0, -10, 10),
    display_mode="ortho",
)
show()

In [ ]:
# Visualize the brain mask (region of interest)
mask_img = nib.load(mask_filename)

print("Visual cortex mask (ventral temporal region for object recognition):\n")
plotting.plot_roi(
    mask_img,
    bg_img=mean_img,
    title="Visual Cortex Mask (Ventral Temporal)",
    cut_coords=(0, -55, -10),
    display_mode="ortho",
)
show()

# Count voxels in mask
mask_data = mask_img.get_fdata()
n_voxels = int(mask_data.sum())
print(f"\nVoxels in visual cortex mask: {n_voxels:,}")
print(f"Percentage of brain: {100 * n_voxels / np.prod(mask_data.shape):.1f}%")

In [ ]:
# Analyze the experimental design
print("=== Experimental Design ===")
print("\nStimulus presentation counts:")
print(labels["labels"].value_counts().sort_index())

# Visualize stimulus distribution
fig, ax = plt.subplots(figsize=(10, 6))
labels["labels"].value_counts().sort_index().plot(kind="bar", ax=ax, color="steelblue")
ax.set_xlabel("Stimulus Category", fontweight="bold")
ax.set_ylabel("Number of Volumes", fontweight="bold")
ax.set_title("Stimulus Presentation Counts", fontsize=14, fontweight="bold")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

print("\n✓ Each category presented ~12 times across the experiment")

## 3. Feature Extraction with Brain Masker

In [ ]:
# Create a masker to extract time series from ROI
# This converts 4D fMRI (x, y, z, time) to 2D matrix (time, voxels)
masker = NiftiMasker(
    mask_img=mask_filename,
    standardize=True,  # Z-score each voxel's time series
    detrend=True,  # Remove linear trends
    smoothing_fwhm=4,  # Spatial smoothing (4mm)
)

print("Extracting features from visual cortex...")
X = masker.fit_transform(fmri_filename)

print("\n✓ Feature matrix extracted!")
print(f"  Shape: {X.shape}")
print(f"  Interpretation: {X.shape[0]} time points × {X.shape[1]} voxels")
print("\nEach row = brain state at one time point")
print("Each column = activity level in one voxel")

In [ ]:
# Prepare labels for classification
# Remove 'rest' periods and 'scrambled' stimuli for cleaner classification
y = labels["labels"].values
conditions_to_use = ["face", "house", "cat", "bottle", "scissors", "shoe", "chair"]
condition_mask = np.isin(y, conditions_to_use)

X_filtered = X[condition_mask]
y_filtered = y[condition_mask]
runs = labels["chunks"].values[condition_mask]  # Run number for cross-validation

print("Filtered dataset for classification:")
print(f"  Samples: {X_filtered.shape[0]}")
print(f"  Features: {X_filtered.shape[1]}")
print(f"  Categories: {len(np.unique(y_filtered))}")
print(f"\nCategories: {list(np.unique(y_filtered))}")

## 4. Brain State Classification

In [ ]:
# Train a Support Vector Machine (SVM) classifier
# SVM is standard for fMRI decoding due to high dimensionality
print("Training SVM classifier for brain state decoding...")
print("(This may take 1-2 minutes)\n")

# Use Leave-One-Run-Out cross-validation (best practice for fMRI)
# Never test on data from the same run as training (prevents overfitting)
cv = LeaveOneGroupOut()
clf = SVC(kernel="linear", C=1.0)

# Compute cross-validated accuracy
scores = cross_val_score(
    clf, X_filtered, y_filtered, cv=cv, groups=runs, scoring="accuracy", n_jobs=-1
)

print("=== Classification Results ===")
print(f"\nCross-validated accuracy: {scores.mean():.2%} (± {scores.std():.2%})")
print(f"Per-fold accuracies: {[f'{s:.2%}' for s in scores]}")
print(f"\nChance level (random guessing): {1 / len(conditions_to_use):.2%}")
print(f"Performance above chance: {scores.mean() - 1 / len(conditions_to_use):.2%}")

if scores.mean() > 0.5:
    print(
        "\n✓ Brain decoding SUCCESSFUL! We can predict what the person is viewing from brain activity alone!"
    )
else:
    print("\n⚠️  Accuracy is modest. May need more data or feature engineering.")

In [ ]:
# Train final model on all data for detailed analysis
from sklearn.model_selection import cross_val_predict

print("Generating predictions for confusion matrix...")
y_pred = cross_val_predict(clf, X_filtered, y_filtered, cv=cv, groups=runs, n_jobs=-1)

# Confusion matrix
cm = confusion_matrix(y_filtered, y_pred, labels=conditions_to_use)
cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

# Visualize confusion matrix
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    cm_normalized,
    annot=True,
    fmt=".2f",
    cmap="Blues",
    xticklabels=conditions_to_use,
    yticklabels=conditions_to_use,
    cbar_kws={"label": "Accuracy"},
    ax=ax,
)
ax.set_xlabel("Predicted Category", fontweight="bold", fontsize=12)
ax.set_ylabel("True Category", fontweight="bold", fontsize=12)
ax.set_title("Brain State Classification Confusion Matrix", fontweight="bold", fontsize=14)
plt.tight_layout()
plt.show()

print("\n=== Per-Category Performance ===")
print(
    classification_report(
        y_filtered, y_pred, labels=conditions_to_use, target_names=conditions_to_use
    )
)

## 5. Brain Activity Visualization

In [ ]:
# Compute average activation for each category
print("Computing category-specific activation patterns...\n")

# Select two contrasting categories
category_1 = "face"
category_2 = "house"

mask_1 = y == category_1
mask_2 = y == category_2

mean_activation_1 = X[mask_1].mean(axis=0)
mean_activation_2 = X[mask_2].mean(axis=0)

# Compute contrast (difference between categories)
contrast = mean_activation_1 - mean_activation_2

# Transform back to brain space
contrast_img = masker.inverse_transform(contrast)

print(f"Visualizing contrast: {category_1.upper()} vs {category_2.upper()}\n")
plotting.plot_stat_map(
    contrast_img,
    bg_img=mean_img,
    title=f"{category_1.capitalize()} vs {category_2.capitalize()} Activation",
    cut_coords=(40, -55, -10),
    display_mode="ortho",
    threshold=0.5,
    cmap="cold_hot",
)
show()

print("\nInterpretation:")
print("  - Red/warm colors: More active for FACES")
print("  - Blue/cool colors: More active for HOUSES")
print("  - Fusiform Face Area (FFA): Known to respond preferentially to faces")
print("  - Parahippocampal Place Area (PPA): Known to respond to scenes/houses")

In [ ]:
# Glass brain visualization (shows activation throughout brain)
print("Glass brain visualization (3D projection):\n")

plotting.plot_glass_brain(
    contrast_img,
    title=f"{category_1.capitalize()} > {category_2.capitalize()}",
    threshold=0.5,
    colorbar=True,
    plot_abs=False,
    cmap="cold_hot",
)
show()

print("\n✓ This 'glass brain' shows where in the brain we see differential activation")
print("  Helps identify specific regions involved in category discrimination")

## 6. Feature Importance Analysis

In [ ]:
# Train a final model to extract feature weights
print("Analyzing which brain regions are most important for classification...\n")

# Focus on face vs house for interpretability
binary_mask = np.isin(y, [category_1, category_2])
X_binary = X[binary_mask]
y_binary = (y[binary_mask] == category_1).astype(int)

# Train linear SVM
clf_binary = SVC(kernel="linear", C=1.0)
clf_binary.fit(X_binary, y_binary)

# Extract weights (coefficients)
weights = clf_binary.coef_[0]

# Transform to brain space
weights_img = masker.inverse_transform(weights)

print(f"Voxel weights for {category_1} vs {category_2} classification:\n")
plotting.plot_stat_map(
    weights_img,
    bg_img=mean_img,
    title=f"SVM Weights: {category_1.capitalize()} vs {category_2.capitalize()}",
    cut_coords=(40, -55, -10),
    display_mode="ortho",
    threshold="auto",
    cmap="cold_hot",
)
show()

print("\nInterpretation:")
print("  - Positive weights (red): Voxels that predict FACES")
print("  - Negative weights (blue): Voxels that predict HOUSES")
print("  - Magnitude indicates importance for classification")
print("\n✓ These patterns align with known neuroscience!")
print("  FFA (fusiform face area) and PPA (parahippocampal place area) are visible")

## 7. Key Findings Summary

In [ ]:
# Generate summary statistics
print("=" * 70)
print("fMRI BRAIN DECODING SUMMARY")
print("=" * 70)

print("\n📊 DATASET:")
print("   • Subjects: 1 (Haxby 2001)")
print(f"   • Categories: {len(conditions_to_use)} visual object types")
print(f"   • fMRI volumes: {X_filtered.shape[0]} (after filtering)")
print(f"   • Voxels analyzed: {X_filtered.shape[1]:,} (visual cortex)")
print("   • Spatial resolution: 3.5mm isotropic")

print("\n🧠 CLASSIFICATION PERFORMANCE:")
print(f"   • Cross-validated accuracy: {scores.mean():.1%} (± {scores.std():.1%})")
print(f"   • Chance level: {1 / len(conditions_to_use):.1%}")
print(f"   • Above chance: {scores.mean() - 1 / len(conditions_to_use):.1%}")
print("   • Method: Linear SVM with leave-one-run-out CV")

print("\n🔬 NEUROSCIENCE INSIGHTS:")
print("   • Successfully decoded visual categories from brain activity")
print("   • Category-selective regions identified (FFA for faces, PPA for places)")
print("   • Results consistent with established neuroscience literature")
print("   • Demonstrates brain represents different object categories in distinct patterns")

print("\n⚙️  METHODS:")
print("   • Preprocessing: Standardization, detrending, smoothing (4mm)")
print("   • Feature extraction: Voxel-wise BOLD signal in visual cortex")
print("   • Classification: Linear Support Vector Machine (SVM)")
print("   • Validation: Leave-one-run-out cross-validation")

print("\n✅ CONCLUSION:")
print("   Machine learning can reliably decode what a person is viewing")
print("   from patterns of brain activity in visual cortex. This demonstrates")
print("   that different visual categories evoke distinct, consistent neural")
print("   representations that can be detected and classified.")

print("=" * 70)

## What You Learned

In just 10-30 minutes, you:

1. Loaded and explored fMRI brain imaging data (NIfTI format)
2. Visualized brain anatomy and regions of interest
3. Extracted features from BOLD signals in visual cortex
4. Trained a classifier to decode cognitive states from brain activity
5. Achieved above-chance classification accuracy
6. Identified brain regions important for visual category recognition
7. Validated results with proper cross-validation
8. Created publication-quality brain visualizations

## Next Steps

### Ready for More?

**Tier 1: SageMaker Studio Lab (4-8 hours, free)**
- Multi-subject ensemble analysis with 10GB data
- Functional connectivity mapping
- Deep learning with 3D CNNs
- Persistent storage for large datasets
- Train models for 5-6 hours continuously

**Tier 2: AWS Starter (4-8 hours, $5-15)**
- Store neuroimaging data in S3
- Distributed preprocessing with AWS Batch
- Managed training on SageMaker
- Multi-cohort analysis (ABIDE, HCP)

**Tier 3: Production Infrastructure (1-2 weeks, $50-500/month)**
- Multi-site neuroimaging datasets (500GB-1TB)
- Distributed deep learning training
- Real-time brain decoding pipelines
- Automated quality control and preprocessing

## Learn More

- **Nilearn Documentation:** [https://nilearn.github.io/](https://nilearn.github.io/)
- **Human Connectome Project:** [https://www.humanconnectome.org/](https://www.humanconnectome.org/)
- **ABIDE Dataset:** [http://fcon_1000.projects.nitrc.org/indi/abide/](http://fcon_1000.projects.nitrc.org/indi/abide/)
- **fMRI Decoding Tutorial:** [https://nilearn.github.io/stable/auto_examples/index.html](https://nilearn.github.io/stable/auto_examples/index.html)

---

**Built with [Claude Code](https://claude.com/claude-code)**